In [ ]:
import numpy as np
import cv2 as cv
import os
import json
import matplotlib.pyplot as plt
import apriltag

import lac.params as params
from lac.perception.pnp import get_tag_corners_world

%load_ext autoreload
%autoreload 2

# Loading fiducial locations


In [ ]:
# known fiducial corner positions
fiducial_group_centers = json.load(
    open(os.path.expanduser("~/LunarAutonomyChallenge/docs/geometry.json"))
)["lander"]["fiducials"]
centers_group_a = fiducial_group_centers["a"]
centers_group_b = fiducial_group_centers["b"]
centers_group_c = fiducial_group_centers["c"]
centers_group_d = fiducial_group_centers["d"]

FIDUCIAL_NAMES = ["top left", "top right", "lower right", "lower left"]


def fiducial_centers_array(centers: dict) -> np.ndarray:
    center_pts = []
    for name in FIDUCIAL_NAMES:
        center_pts.append(np.array([centers[name]["x"], centers[name]["y"], centers[name]["z"]]))
    return np.array(center_pts)

In [ ]:
tag_group_yaw_angles = {
    "a": 135,
    "b": 45,
    "c": 315,
    "d": 225,
}

In [ ]:
tag_centers = {}

for group, group_vals in fiducial_group_centers.items():
    for tag, tag_vals in group_vals.items():
        tag_centers[tag_vals["id"]] = np.array([tag_vals["x"], tag_vals["y"], tag_vals["z"]])

In [ ]:
tag_centers

# AprilTag detection


In [ ]:
data_path = os.path.expanduser("~/LunarAutonomyChallenge/output/lander_closeups/")
CAM_NAME = "FrontLeft"
imgs = os.listdir(os.path.join(data_path, CAM_NAME))
imgs = [os.path.join(data_path, CAM_NAME, imgs[i]) for i in range(0, len(imgs), 1)]

In [ ]:
i = 2019
img = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)
plt.imshow(img, cmap="gray")

In [ ]:
gray = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"), cv.IMREAD_GRAYSCALE)
options = apriltag.DetectorOptions(families="tag36h11")
detector = apriltag.Detector(options)
results = detector.detect(img)

In [ ]:
results

In [ ]:
results[0].tag_id

In [ ]:
get_tag_corners_world(tag_centers[results[0].tag_id])

In [ ]:
results[-1].corners

From testing once this detector seems worse (and also only gives integer pixel values)


In [ ]:
import cv2.aruco as aruco

# Define AprilTag dictionary
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_APRILTAG_36h11)
parameters = aruco.DetectorParameters()
detector = aruco.ArucoDetector(aruco_dict, parameters)

# Read and preprocess image
image = cv.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"))
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

# Detect markers
corners, ids, _ = detector.detectMarkers(gray)

# Solving for camera pose using PnP


In [ ]:
import cv2
import numpy as np
import cv2.aruco as aruco
# Load camera parameters

camera_matrix = params.CAMERA_INTRINSICS
dist_coeffs = np.zeros(5)  # Replace with actual values

# Define AprilTag dictionary
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_APRILTAG_36h11)
parameters = aruco.DetectorParameters()
detector = aruco.ArucoDetector(aruco_dict, parameters)

# Read and preprocess image
image = cv2.imread(os.path.join(data_path, CAM_NAME, f"{i}.png"))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect markers
corners, ids, _ = detector.detectMarkers(gray)


if ids is not None:
    tag_size = 0.339  # Set tag size in meters
    # New way to estimate pose in OpenCV 4.7+
    retval, rvecs, tvecs = cv2.solvePnP(
        objectPoints=np.array(
            [
                [-tag_size / 2, tag_size / 2, 0],  # Top-left
                [tag_size / 2, tag_size / 2, 0],  # Top-right
                [tag_size / 2, -tag_size / 2, 0],  # Bottom-right
                [-tag_size / 2, -tag_size / 2, 0],  # Bottom-left
            ],
            dtype=np.float32,
        ),
        imagePoints=results[-1].corners,  # First detected marker’s corners
        cameraMatrix=camera_matrix,
        distCoeffs=dist_coeffs,
        flags=cv2.SOLVEPNP_IPPE_SQUARE,
    )
    if retval:
        print("Rotation Vector (rvec):", rvecs)
        print("Translation Vector (tvec):", tvecs)
        # Convert rotation vector to rotation matrix
        R, _ = cv2.Rodrigues(rvecs)
        # Compute homogeneous transformation matrix
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = tvecs.flatten()
        print("Homogeneous Transformation Matrix:\n", T)
        # Compute distance
        distance = np.linalg.norm(tvecs)  # Euclidean distance
        print(f"Distance to AprilTag: {distance:.2f} meters")

In [ ]:
ids

In [ ]:
corners

In [ ]:
from scipy.spatial.transform import Rotation

Rotation.from_matrix(R).as_euler("xyz", degrees=True)

In [ ]:
from lac.utils.frames import invert_transform_mat


T_inv = invert_transform_mat(T)
print(np.round(T_inv, 2))
print(Rotation.from_matrix(T_inv[:3, :3]).as_euler("xyz", degrees=True))